In [1]:
"""
1.651.119 -> Στο σύνολο γραμμές καταχωρημένων ενδείξεων.
752.146 -> Ομαδοποιημένες εγγραφές στο 8ώρο.
1.547 -> Eισαγωγές στις ΜΕΘ
1.662 -> Ομαδοποιημένες εγγραφές στο 8ώρο που αφορούν ασθενείς από τα επείγοντα στις ΜΕΘ
"""

'\n1.651.119 -> Στο σύνολο γραμμές καταχωρημένων ενδείξεων.\n752.146 -> Ομαδοποιημένες εγγραφές στο 8ώρο.\n1.547 -> Eισαγωγές στις ΜΕΘ\n1.662 -> Ομαδοποιημένες εγγραφές στο 8ώρο που αφορούν ασθενείς από τα επείγοντα στις ΜΕΘ\n'

In [2]:
import pandas as pd

In [3]:
df_vitalsign = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\ed\Vitalsign.csv.gz')
df_ed_to_icu = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o16_ed_patients_to_icu.csv', usecols = ['subject_id', 'hadm_id','stay_id'])

In [4]:
# Μετατροπή της στήλης charttime από object σε datetime
df_vitalsign['charttime']= pd.to_datetime(df_vitalsign['charttime'])

In [5]:
# Ομαδοποίηση καταγραφών ανά στήλη στο 8ώρο.

grouped_multiple = (df_vitalsign.groupby(['subject_id', 'stay_id',
                                          pd.Grouper(key='charttime',
                                                     freq='8H')]).agg({'temperature': ['mean'],
                                                                       'heartrate': ['mean'],
                                                                       'resprate': ['mean'],
                                                                       'o2sat': ['mean'],
                                                                       'sbp': ['mean'],
                                                                       'dbp': ['mean']}))
grouped_multiple.columns = (['Mean_temperature',
                             'Mean_heartrate',
                             'Mean_resprate',
                             'Mean_o2sat',
                             'Mean_sbp',
                             'Mean_dbp'])
grouped_multiple = grouped_multiple.reset_index()

In [6]:
# Κενές τιμές στον πίνακα
grouped_multiple.isna().sum()

subject_id              0
stay_id                 0
charttime               0
Mean_temperature    60071
Mean_heartrate       6126
Mean_resprate        7643
Mean_o2sat          28566
Mean_sbp             7394
Mean_dbp             7394
dtype: int64

In [7]:
df_ed_to_icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   subject_id  1547 non-null   int64
 1   hadm_id     1547 non-null   int64
 2   stay_id     1547 non-null   int64
dtypes: int64(3)
memory usage: 36.4 KB


In [8]:
rslt_df=pd.merge(df_ed_to_icu, grouped_multiple, on=['stay_id'])

In [9]:
rslt_df = rslt_df.drop(['subject_id_y'], axis=1)

In [10]:
rslt_df = rslt_df.rename(columns={"subject_id_x": "subject_id"})

In [11]:
rslt_df.insert(0, 'Department', 'ed')

In [13]:
#Εξαγωγή σε csv
rslt_df.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o17_ed_to_icu_vitalsigns.csv', index=False)